# Spark 3

In [1]:
import os
import sys
spark_home = os.environ.get('SPARK_HOME', None)
sys.path.insert(0, spark_home + "python")
os.environ["SPARK_LOCAL_IP"]='localhost'
from pyspark import SparkContext, SparkConf#, HiveContext
conf = SparkConf()\
             .setAppName("Example Spark")\
             .setMaster("local[2]")\
             .setAppName("CountingSheep")\
             .set("spark.sql.catalogImplementation", "hive")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/29 01:38:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 3.5.6
      /_/

Using Python version 3.13.3 (main, Jun 16 2025 18:15:32)
Spark context Web UI available at http://localhost:4040
Spark context available as 'sc' (master = local[2], app id = local-1751150319437).
SparkSession available as 'spark'.


In [3]:
spark

In [4]:
from pyspark.sql.types import *
import pyspark.sql.functions as f_
from pyspark.sql import Window
from pyspark.sql import DataFrame
from pyspark.sql.functions import udf
import pandas as pd

# Example DataFrame

In [5]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(rowno=1, double_value=3., str_value='Value 1', date_value=date(2004, 5, 9), timestamp_value=datetime(2002, 1, 1, 12, 0)),
    Row(rowno=2, double_value=4., str_value='Value 2', date_value=date(2004, 5, 2), timestamp_value=datetime(2002, 1, 2, 15, 15)),
    Row(rowno=3, double_value=5., str_value='Value 3', date_value=date(2007, 3, 1), timestamp_value=datetime(2002, 1, 3, 19, 23))
])

In [6]:
df.show()

+-----+------------+---------+----------+-------------------+
|rowno|double_value|str_value|date_value|    timestamp_value|
+-----+------------+---------+----------+-------------------+
|    1|         3.0|  Value 1|2004-05-09|2002-01-01 12:00:00|
|    2|         4.0|  Value 2|2004-05-02|2002-01-02 15:15:00|
|    3|         5.0|  Value 3|2007-03-01|2002-01-03 19:23:00|
+-----+------------+---------+----------+-------------------+



In [7]:
df.dtypes

[('rowno', 'bigint'),
 ('double_value', 'double'),
 ('str_value', 'string'),
 ('date_value', 'date'),
 ('timestamp_value', 'timestamp')]

# Local DataBases

In [8]:
spark.sql("drop database if exists my_db cascade")
spark.sql("create database my_db")

DataFrame[]

In [9]:
spark.sql("show databases").show()

+---------+
|namespace|
+---------+
|  default|
|    my_db|
+---------+



# Save

In [10]:
df.write.format("parquet").mode("overwrite").saveAsTable("default.my_table")

In [11]:
df.write.format("parquet").mode("overwrite").saveAsTable("my_db.new_my_table")

In [12]:
df2 = spark.table("default.my_table")

In [13]:
df2.show()

+-----+------------+---------+----------+-------------------+
|rowno|double_value|str_value|date_value|    timestamp_value|
+-----+------------+---------+----------+-------------------+
|    1|         3.0|  Value 1|2004-05-09|2002-01-01 12:00:00|
|    2|         4.0|  Value 2|2004-05-02|2002-01-02 15:15:00|
|    3|         5.0|  Value 3|2007-03-01|2002-01-03 19:23:00|
+-----+------------+---------+----------+-------------------+



In [14]:
df3 = spark.table("my_db.new_my_table")

In [15]:
df3.show()

+-----+------------+---------+----------+-------------------+
|rowno|double_value|str_value|date_value|    timestamp_value|
+-----+------------+---------+----------+-------------------+
|    1|         3.0|  Value 1|2004-05-09|2002-01-01 12:00:00|
|    2|         4.0|  Value 2|2004-05-02|2002-01-02 15:15:00|
|    3|         5.0|  Value 3|2007-03-01|2002-01-03 19:23:00|
+-----+------------+---------+----------+-------------------+



In [16]:
spark.sql("show tables from default").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default| my_table|      false|
+---------+---------+-----------+



In [17]:
df4 = spark.table("my_db.new_my_table")

In [18]:
df4.where(f_.col("rowno")==1).show()

+-----+------------+---------+----------+-------------------+
|rowno|double_value|str_value|date_value|    timestamp_value|
+-----+------------+---------+----------+-------------------+
|    1|         3.0|  Value 1|2004-05-09|2002-01-01 12:00:00|
+-----+------------+---------+----------+-------------------+



In [19]:
spark.sql("show tables from my_db").show()

+---------+------------+-----------+
|namespace|   tableName|isTemporary|
+---------+------------+-----------+
|    my_db|new_my_table|      false|
+---------+------------+-----------+



In [20]:
spark.stop()